## 门控循环单元（GRU）— 从0开始
上一节中，我们介绍了循环神经网络中的梯度计算方法。我们发现，循环神经网络的隐含层变量梯度可能会出现衰减或爆炸。虽然梯度裁剪可以应对梯度爆炸，但无法解决梯度衰减的问题。因此，给定一个时间序列，例如文本序列，循环神经网络在实际中其实较难捕捉两个时刻距离较大的文本元素（字或词）之间的依赖关系。

门控循环神经网络（gated recurrent neural networks）的提出，是为了更好地捕捉时序数据中间隔较大的依赖关系。其中，门控循环单元（gated recurrent unit，简称GRU）是一种常用的门控循环神经网络。它由Cho、van Merrienboer、 Bahdanau和Bengio在2014年被提出。

### 门控循环单元
我们先介绍门控循环单元的构造。它比循环神经网络中的隐含层构造稍复杂一点。

### 重置门和更新门
门控循环单元的隐含状态只包含隐含层变量$H$。假定隐含状态长度为$h$，给定时刻t的一个样本数为$n$特征向量维度为$x$的批量数据$\mathbf{X}_t \in \mathbb{R}^{n \times x}$和上一时刻隐含状态$\mathbf{X}_t \in \mathbb{R}^{n \times x}$，重置门（reset gate）$\mathbf{R}_t \in \mathbb{R}^{n \times h}$和更新门（update gate）$\mathbf{Z}_t \in \mathbb{R}^{n \times h}$的定义如下：

$\mathbf{R}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xr} + \mathbf{H}_{t-1} \mathbf{W}_{hr} + \mathbf{b}_r)$

$\mathbf{Z}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xz} + \mathbf{H}_{t-1} \mathbf{W}_{hz} + \mathbf{b}_z)$

其中的$\mathbf{W}_{xr}, \mathbf{W}_{xz} \in \mathbb{R}^{x \times h}$,和$\mathbf{W}_{hr}, \mathbf{W}_{hz} \in \mathbb{R}^{h \times h}$是可学习的权重参数，$\mathbf{b}_r, \mathbf{b}_z \in \mathbb{R}^{1 \times h}$是可学习的偏移参数。函数$\sigma$自变量中的三项相加使用了广播。

需要注意的是，重置门和更新门使用了值域为$[0,1]$的函数$\sigma(x) = 1/(1+\text{exp}(-x))$。因此，重置门$\mathbf{R}_t$和更新门$\mathbf{Z}_t$中每个元素的值域都是$[0,1]$。

### 候选隐含状态
我们可以通过元素值域在$[0,1]$的更新门和重置门来控制隐含状态中信息的流动：这通常可以应用按元素乘法符$\odot$。门控循环单元中的候选隐含状态$\tilde{\mathbf{H}}_t \in \mathbb{R}^{n \times h}$使用了值域在$[−1,1]$的双曲正切函数$\tanh$做激活函数：

$\tilde{\mathbf{H}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{R}_t \odot \mathbf{H}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h)$

其中的$\mathbf{W}_{xh} \in \mathbb{R}^{x \times h}$和$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$是可学习的权重参数，$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$是可学习的偏移参数。

需要注意的是，候选隐含状态使用了重置门来控制包含过去时刻信息的上一个隐含状态的流入。如果重置门近似0，上一个隐含状态将被丢弃。因此，重置门提供了丢弃与未来无关的过去隐含状态的机制。

### 隐含状态
隐含状态$\mathbf{H}_t \in \mathbb{R}^{n \times h}$的计算使用更新门$\mathbf{Z}_t$来对上一时刻的隐含状态$\mathbf{H}_{t-1}$和当前时刻的候选隐含状态$\tilde{\mathbf{H}}_t$做组合，公式如下：

$\mathbf{H}_t = \mathbf{Z}_t \odot \mathbf{H}_{t-1}  + (1 - \mathbf{Z}_t) \odot \tilde{\mathbf{H}}_t$

需要注意的是，更新门可以控制过去的隐含状态在当前时刻的重要性。如果更新门一直近似1，过去的隐含状态将一直通过时间保存并传递至当前时刻。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时序数据中间隔较大的依赖关系。

我们对门控循环单元的设计稍作总结：

重置门有助于捕捉时序数据中短期的依赖关系。
更新门有助于捕捉时序数据中长期的依赖关系。
输出层的设计可参照循环神经网络中的描述。

### 实验
为了实现并展示门控循环单元，我们依然使用周杰伦歌词数据集来训练模型作词。这里除门控循环单元以外的实现已在循环神经网络中介绍。

### 数据处理
我们先读取并对数据集做简单处理。

In [4]:
import zipfile
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip', 'r') as zin:
    zin.extractall('../../data/')

with open('../../data/jaychou_lyrics.txt') as f:
    corpus_chars = f.read().decode('utf-8')

corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:20000]

idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
corpus_indices = [char_to_idx[char] for char in corpus_chars]

vocab_size = len(char_to_idx)
print('vocab size:', vocab_size)

('vocab size:', 1465)


我们使用onehot来将字符索引表示成向量。



In [21]:
import numpy as np

def one_hot(positions, vocal_size):
    if len(positions.shape) == 0:
        positions = np.expand_dims(positions, 0)
        positions = np.expand_dims(positions, 1)
    if len(positions.shape) == 1:
        positions = np.expand_dims(positions, 1)
    all_zeros = np.zeros((positions.shape[0], vocab_size))
    for i in xrange(positions.shape[0]):
        for j in xrange(positions.shape[1]):
            all_zeros[i, int(positions[i, j])] = 1
        
    return all_zeros
        
one_hot(np.array([0, 2]), vocab_size)

def get_inputs(data):
    return [one_hot(X, vocab_size) for X in data.T]

### 初始化模型参数
以下部分对模型参数进行初始化。参数hidden_dim定义了隐含状态的长度。

In [6]:
import tensorflow as tf

input_dim = vocab_size
# 隐含状态长度
hidden_dim = 256
output_dim = vocab_size
weight_scale = .01


with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
    W_xz = tf.get_variable(name='weights_xz', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hz = tf.get_variable(name='weights_hz', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_z = tf.get_variable(name='bias_z', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)

    W_xr = tf.get_variable(name='weights_xr', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hr = tf.get_variable(name='weights_hr', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_z = tf.get_variable(name='bias_r', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    
    W_xh = tf.get_variable(name='weights_xh', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hh = tf.get_variable(name='weights_hh', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_h = tf.get_variable(name='bias_h', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    

    W_hy = tf.get_variable(name='weights_output', shape=[hidden_dim, output_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_y = tf.get_variable(name='bias_output', shape=[output_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)


params = [W_xz, W_hz, b_z, W_xr, W_hr, b_z, W_xh, W_hh, b_h, W_hy, b_y]


### 定义模型
我们将前面的模型公式翻译成代码。

In [39]:
def gru_rnn(inputs, H, *params):
    # inputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵
    # H: 尺寸为 batch_size * hidden_dim 矩阵
    # outputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵

    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hy, b_y = params[0]
    outputs = []
    num_steps = inputs.get_shape().as_list()[0]

    for i in range(num_steps):
        X = inputs[i]
        Z = tf.nn.sigmoid(tf.matmul(X, W_xz) + tf.matmul(H, W_hz) + b_z)
        R = tf.nn.sigmoid(tf.matmul(X, W_xr) + tf.matmul(H, W_hr) + b_r)
        H_tilda = tf.nn.tanh(tf.matmul(X, W_xh) + R * tf.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda 
        Y = tf.matmul(H, W_hy) + b_y
        outputs.append(Y)
    return (outputs, H)

### 训练模型
下面我们开始训练模型。我们假定谱写歌词的前缀分别为“分开”、“不分开”和“战争中部队”。这里采用的是相邻批量采样实验门控循环单元谱写歌词。



In [40]:
import random
import numpy as np

def data_iter_random(corpus_indices, batch_size, num_steps):
    # 减一是因为label的索引是相应data的索引加一
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    # 随机化样本
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回num_steps个数据
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        data = np.array(
            [_data(j * num_steps) for j in batch_indices])
        label = np.array(
            [_data(j * num_steps + 1) for j in batch_indices])
        yield data.astype(np.float32), label

In [41]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = np.array((corpus_indices))
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size

    indices = corpus_indices[0: batch_size * batch_len].reshape((
        batch_size, batch_len))
    # 减一是因为label的索引是相应data的索引加一
    epoch_size = (batch_len - 1) // num_steps

    for i in range(epoch_size):
        i = i * num_steps
        data = indices[:, i: i + num_steps]
        label = indices[:, i + 1: i + num_steps + 1]
        yield data, label

In [42]:
slim = tf.contrib.slim

learning_rate = 1e-1
max_steps = 10000
batch_size = 32
train_loss = 0.0
train_acc = 0.0
is_random_iter=True
epochs=40
num_steps=35 
learning_rate=1e-2
batch_size=32

#训练
print hidden_dim
num_inputs = num_outputs = vocab_size
input_placeholder = tf.placeholder(tf.float32, [num_steps, None, num_inputs])
state_h_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])

state_c_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])
gt_placeholder = tf.placeholder(tf.int64, [num_steps, None, 1])



outputs, state_h = gru_rnn(input_placeholder, state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)

loss = tf.losses.sparse_softmax_cross_entropy(logits=outputs,  labels=tf.reshape(gt_placeholder, (num_steps*batch_size, 1)))

var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name
op = tf.train.AdamOptimizer(learning_rate)

gradients = tf.gradients(loss, params)

#process gradients
clipped_gradients, norm = tf.clip_by_global_norm(gradients, 1)

train_op = op.apply_gradients(zip(clipped_gradients, params))

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

if is_random_iter:
    data_iter = data_iter_random
else:
    data_iter = data_iter_consecutive
for e in range(0, epochs):
    # 如使用相邻批量采样，在同一个epoch中，隐含变量只需要在该epoch开始的时候初始化。
    if not is_random_iter:
        state_h_init = np.zeros(shape=(batch_size, hidden_dim))
    train_loss, num_examples = 0, 0

    for data, label in data_iter(corpus_indices, batch_size, num_steps):
        # 如使用随机批量采样，处理每个随机小批量前都需要初始化隐含变量。
        if is_random_iter:
            state_h_init = np.zeros(shape=(batch_size, hidden_dim))
        feed_dict = {input_placeholder: get_inputs(data), state_h_placeholder: state_h_init, gt_placeholder: np.expand_dims(label.T, axis=-1)}
        loss_, state_h_, _ = sess.run([loss, state_h, train_op], feed_dict=feed_dict)
        state_h_init = state_h_

        print np.exp(loss_)


256
rnn/weights_xz
rnn/weights_hz
rnn/bias_z
rnn/weights_xr
rnn/weights_hr
rnn/bias_r
rnn/weights_xh
rnn/weights_hh
rnn/bias_h
rnn/weights_output
rnn/bias_output
1464.9376
1439.6514
1249.856
649.8489
652.86163
511.26956
649.5704
549.7654
530.80414
418.92908
443.1054
455.75653
436.66254
416.843
339.69907
390.52524
495.65125
340.27444
275.5851
306.25516
295.50293
294.7655
283.7825
306.8359
283.94656
302.96762
292.34683
272.07056
231.1862
272.39
266.0971
222.43002
245.96857
224.33475
164.07266
155.38518
164.51215
166.97023
135.26433
135.62885
144.93999
156.54453
145.53488
141.26868
116.46877
122.53087
137.13354
110.45613
90.34166
113.38717
94.81563
79.833435
62.691242
71.12315
57.73551
53.634052
52.863514
58.326344
57.293995
48.356567
46.911655
34.347435
44.95937
46.96556
40.04711
40.3418
57.008533
40.487366
26.308485
28.486073
28.07149
19.740408
25.616179
21.053658
20.104101
23.428736
19.016323
14.829099
21.882864
22.722208
13.322555
17.549547
22.380533
20.338774
16.782362
6.9460087
10.3

In [45]:
epochs = 200
num_steps = 35
learning_rate = 1e-4
batch_size = 32


seq1 = '天空'.decode('utf-8')
seq2 = '天天'.decode('utf-8')
seq3 = '云'.decode('utf-8')
seqs = [seq1, seq2, seq3]
print seq1

天空


In [46]:
pred_len = 100
test_num_steps = 1
# 预测
num_inputs = num_outputs = vocab_size    
test_input_placeholder = tf.placeholder(tf.float32, [test_num_steps, None, num_inputs])
test_state_h_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])
test_state_c_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])


outputs, state_h = gru_rnn(test_input_placeholder, test_state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)


var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name


for seq in seqs:
    prefix = seq

    prefix = prefix.lower()
    test_state_h_init = np.zeros((1, hidden_dim))
    output_seq = [char_to_idx[prefix[0]]]
    for i in range(pred_len + len(prefix)):
        X = np.array([output_seq[-1]])
        # 在序列中循环迭代隐含变量。
        Y, state_h_ = sess.run([outputs, state_h], feed_dict={test_input_placeholder: get_inputs(X), test_state_h_placeholder: test_state_h_init})
        test_state_h_init = state_h_
        if i < len(prefix)-1:
            next_input = char_to_idx[prefix[i+1]]
        else:
            next_input = np.argmax(Y[0])
        output_seq.append(next_input)
    print ''.join([idx_to_char[i] for i in output_seq])

    print()

rnn/weights_xz
rnn/weights_hz
rnn/bias_z
rnn/weights_xr
rnn/weights_hr
rnn/bias_r
rnn/weights_xh
rnn/weights_hh
rnn/bias_h
rnn/weights_output
rnn/bias_output
天空叫我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 
()
天天看着对我有属于我的天 我要一步一步往上爬 在最高点乘着叶片往前飞 任风吹干流过的泪和汗 总有一天我有属于我的天 我要一步一步往上爬 在最高点乘着叶片往前飞 任风吹干流过的泪和汗 总有一天我有属于我的天 
()
云在我面前 捏成你的形状 随风跟著我 一口一口吃掉忧愁 我出的那天我看着它一定实现 它一定实现 娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口默默等著我 娘子依旧每日折一枝杨柳 你在那里 
()
